In [2]:
import numpy
import pickle
from collections import OrderedDict
import json
import zmq
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy
import sys
from uuid import uuid4
import uuid
import warnings
warnings.filterwarnings('ignore')
numpy.set_printoptions(suppress=False)
torch.set_printoptions(sci_mode=False)
import pickle
import base64

# Assuming UTF-8 encoding, change to something else if you need to
base64.b64encode("password".encode("utf-8"))


def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

def is_pickle_stream(stream):
    try:
        pickle.loads(stream)
        return True
    except:
        return False

context2 = zmq.Context()
# print("Connecting to key server ...")
socket = context2.socket(zmq.DEALER)
socket.connect("tcp://localhost:5555")
identity = str(id)
socket.identity = identity.encode("ascii")
    


context1 = zmq.Context()
print("Connecting to server…")
socket1 = context1.socket(zmq.DEALER)
socket1.connect("tcp://localhost:5556")
identity = str(id)
socket1.identity = identity.encode("ascii")
    
    
sub_socket = context2.socket(zmq.SUB)
sub_socket.connect("tcp://localhost:5557")
sub_socket.setsockopt_string(zmq.SUBSCRIBE, '')



def train(discriminator_arrived):



    import torch
    import torchvision
    from torchvision import datasets, transforms
    import matplotlib.pyplot as plt
    from tqdm.notebook import tqdm
    from torchsummary import summary
    import numpy as np
    import torchvision.utils as vutils
    import time
    from torch import nn, optim
    from torch.nn import functional as F
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST("MNIST_data/", download=True, train=True, transform=transform)
    dataset_range = list(range(0, 3000))
    trainset_range = torch.utils.data.Subset(train_dataset, dataset_range)
    trainloader = torch.utils.data.DataLoader(dataset=trainset_range, 
                                               batch_size=128, 
                                               shuffle=True)

    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Linear') != -1:
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)


    def elapsed_time(start, end):
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Elapsed Time: {:0>2}:{:0>2}:{:05.2f}"
                    .format(int(hours),int(minutes),seconds))

    def elapsed_time_total(start, end):
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Total Traning Time: {:0>2}:{:0>2}:{:05.2f}"
                    .format(int(hours),int(minutes),seconds))

    class FCN(nn.Module):
        def __init__(self):
            super(FCN, self).__init__()
            self.fc1 = nn.Linear(28 * 28, 512)
            self.fc2 = nn.Linear(512, 256)
            self.fc3 = nn.Linear(256, 128)
            self.fc4 = nn.Linear(128, 10)

        def forward(self, x):
            x = x.view(-1, 28 * 28)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = F.log_softmax(self.fc4(x), dim=1)
            return x


    D = FCN().to(device)
    
   ####################################################### 
    
    discriminator_old = D.state_dict()
    shape_d = []
    for name, param in discriminator_old.items():
        
        shape_d.append(param.cpu().numpy())

    valss_d = []
    
    valss_d = discriminator_arrived
    
    #######################################################
    i = 0

    for name, param in discriminator_old.items():

        # Transform the parameter as required.
        transformed_param_d = valss_d[i]

        # Update the parameter.
        if isinstance(transformed_param_d, np.ndarray):
            param.copy_(torch.from_numpy(transformed_param_d))
        else:
            param.copy_(transformed_param_d)
        i = i + 1
        

    #######################################################
    # Training setup
    criterion = nn.NLLLoss().to(device)
    optimizer = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
    EPOCHS = 10

    train_losses = []
    accuracies = []

    start_total = time.time()
    for epoch in range(1, EPOCHS + 1):
        start = time.time()
        running_loss = 0.0
        correct_preds = 0  # To track number of correct predictions
        total_preds = 0  # To track total number of predictions

        for i, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = D(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate the number of correct predictions
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

            if i % 100 == 0:
                print(f"Epoch [{epoch}/{EPOCHS}], Batch [{i}], Loss: {loss.item()}")

        accuracy = 100 * correct_preds / total_preds  # Calculate accuracy for this epoch
        accuracies.append(accuracy)
        train_losses.append(running_loss / len(trainloader))

        print(f"Epoch [{epoch}/{EPOCHS}], Accuracy: {accuracy:.2f}%")  # Print accuracy

        end = time.time()
        elapsed_time(start, end)

    print(f"Overall Accuracy: {sum(accuracies)/len(accuracies):.2f}%")

    end_total = time.time()
    elapsed_time_total(start_total, end_total)


    #######################################################

    model_last_d = D.state_dict()
    
    keys_d = []
    vals_d = []
    
    for k, v in model_last_d.items():
        
        keys_d.append(k)
        vals_d.append(v.cpu())
        
    keys_d = numpy.array(keys_d)
    vals_d = [v.cpu().numpy() for v in vals_d]

    
    cipher_d = vals_d


    #######################################################

    return cipher_d 


for request in range(10):
    
    print("request in loop: " , request)
    
    if request == 0 :
        
        print("request in if" , request)
        message = b"New"
        socket.send(message)

        discriminator = socket.recv()
        discriminator = pickle.loads(discriminator) 
        
    else: 
        print("request in else" , request)
        discriminator = sub_socket.recv()
        print(f"New update number {request} received from the server ")
    
    if is_pickle_stream(discriminator):
        discriminator = pickle.loads(discriminator) 
        print("if train started")
        cipher_d = train(discriminator)
    else: 
        print("else train started")
        cipher_d = train(discriminator)
    print("train finished")
    print(f"Model number {request} locally trained")

    cipher_d = pickle.dumps(cipher_d) 
    socket.send(cipher_d)

Connecting to server…
request in loop:  0
request in if 0
else train started
Epoch [1/10], Batch [0], Loss: 2.3033831119537354
Epoch [1/10], Accuracy: 47.20%
Elapsed Time: 00:00:00.54
Epoch [2/10], Batch [0], Loss: 1.5648516416549683
Epoch [2/10], Accuracy: 69.23%
Elapsed Time: 00:00:00.54
Epoch [3/10], Batch [0], Loss: 0.8454878330230713
Epoch [3/10], Accuracy: 80.60%
Elapsed Time: 00:00:00.54
Epoch [4/10], Batch [0], Loss: 0.663864016532898
Epoch [4/10], Accuracy: 85.20%
Elapsed Time: 00:00:00.54
Epoch [5/10], Batch [0], Loss: 0.52450031042099
Epoch [5/10], Accuracy: 87.10%
Elapsed Time: 00:00:00.54
Epoch [6/10], Batch [0], Loss: 0.4716402590274811
Epoch [6/10], Accuracy: 88.77%
Elapsed Time: 00:00:00.55
Epoch [7/10], Batch [0], Loss: 0.4675552248954773
Epoch [7/10], Accuracy: 89.27%
Elapsed Time: 00:00:00.54
Epoch [8/10], Batch [0], Loss: 0.2645012438297272
Epoch [8/10], Accuracy: 90.13%
Elapsed Time: 00:00:00.84
Epoch [9/10], Batch [0], Loss: 0.34470367431640625
Epoch [9/10], Accur

Epoch [5/10], Accuracy: 86.83%
Elapsed Time: 00:00:01.76
Epoch [6/10], Batch [0], Loss: 0.3242656886577606
Epoch [6/10], Accuracy: 88.37%
Elapsed Time: 00:00:01.67
Epoch [7/10], Batch [0], Loss: 0.6028808355331421
Epoch [7/10], Accuracy: 89.03%
Elapsed Time: 00:00:01.66
Epoch [8/10], Batch [0], Loss: 0.291029155254364
Epoch [8/10], Accuracy: 90.00%
Elapsed Time: 00:00:01.68
Epoch [9/10], Batch [0], Loss: 0.291604220867157
Epoch [9/10], Accuracy: 91.27%
Elapsed Time: 00:00:01.67
Epoch [10/10], Batch [0], Loss: 0.2744159698486328
Epoch [10/10], Accuracy: 91.73%
Elapsed Time: 00:00:01.51
Overall Accuracy: 81.78%
Total Traning Time: 00:00:16.56
train finished
Model number 6 locally trained
request in loop:  7
request in else 7
New update number 7 received from the server 
if train started
Epoch [1/10], Batch [0], Loss: 2.3130624294281006
Epoch [1/10], Accuracy: 45.93%
Elapsed Time: 00:00:01.71
Epoch [2/10], Batch [0], Loss: 1.5738946199417114
Epoch [2/10], Accuracy: 69.80%
Elapsed Time: 00